# How to read and manipulate simulated orbit data

In [ ]:
include("../modules/orbits.jl")

using NCDatasets
using Plots
using Dates

# Read in NetCDF orbit file
ds = Dataset("../inputs/orbit_output_062021.nc");
#ds = Dataset("../inputs/orbitOutput_082020.nc");
        
# read in time and position data
read_samps = 1:15000;
orbit_time = ds["time"][read_samps]*1.0;
orbit_pos  = ds["position"][:,:,read_samps]*1e3;

#use plotly backend
plotly();

# plot the ECI orbit for one of the SCs
plot(orbit_time, orbit_pos[:,1,:]'/1e3, xaxis=("time (sec)"), ylabel=("ECI position (km)"), 
    labels = ["ECI-x" "ECI-y" "ECI-z"])


# Read in the Directional Cosine Matrix (DCM) and convert ECI position to ECEF 

In [ ]:

#read in dcm from orbit file - orbit files aren't guaranteed to have this
dcm = ds["dcm"];

#convert ECI orbit position to ECEF orbit position
orbit_pos_ecef = Orbits.ecef_orbitpos(orbit_pos, dcm);

# plot the ECEF orbit computed using dcm from NetCDF file
plot(orbit_time, orbit_pos[:,1,:]'/1e3, xaxis=("time (sec)"), ylabel=("ECI position (km)"), 
    labels = ["ECI-x" "ECI-y" "ECI-z"])
scatter!(orbit_time[1:100:end], orbit_pos_ecef[:,1,1:100:end]'/1e3, marker = (:+), 
    labels = ["ECEF-x" "ECEF-y" "ECEF-z"])

# Create DCM and convert to ECEF 

In [ ]:

#read in the epoch from file - this is guaranteed to be there in each orbit file
dv = ds.attrib["epoch"];

#convert date variable to DateTime struct
epoch = DateTime(dv[1], dv[2], dv[3], dv[4], dv[5], dv[6]);

#compute DCM
dcm = Orbits.eci_dcm(orbit_time, epoch);

#convert ECI position to ECEF
orbit_pos_ecef2 = Orbits.ecef_orbitpos(orbit_pos, dcm);

# plot the ECEF orbit computed using DCM based on epoch
plot(orbit_time, orbit_pos[:,1,:]'/1e3, xaxis=("time (sec)"), ylabel=("ECI position (km)"), 
    labels = ["ECI-x" "ECI-y" "ECI-z"])
scatter!(orbit_time[1:100:end], orbit_pos_ecef2[:,1,1:100:end]'/1e3, marker = (:circle), 
    labels = ["ECEF-x" "ECEF-y" "ECEF-z"])

# Interpolate Orbit to new timebase

In [ ]:
#create new time vector
fs = 15; #sampling rate (Hz).
duration = 3; #duration of orbit snippet
offset = 12.0; #offset to start of orbit snippet
tv = 0+offset:1/fs:duration+offset; #time vector

#interpolate orbit
orbit_pos_interp = Orbits.interp_orbit(minimum(orbit_time):maximum(orbit_time), orbit_pos, tv);

#plot the original and the interpolate orbit
plot(tv, orbit_pos_interp[:,2,:]'/1e3, xaxis=("time (sec)"), 
    ylabel=("ECEF position (km)"),marker = (:dot), markersize=1)
ind = findall(x->x .>= tv[1] && x .<= tv[end], orbit_time);
scatter!(orbit_time[ind], orbit_pos[:,1,ind]'/1e3, marker = (:square))

# Compute perpendicular baselines

In [ ]:
look_angle = 30.0; #look angle in degrees

#read ECI velocity
orbit_pos  = ds["position"][:,:,read_samps]*1e3;
orbit_vel = ds["velocity"][:,:,read_samps]*1e3;

# convert to ECEF
dcm = Orbits.eci_dcm(orbit_time, epoch);
pos_ecef, vel_ecef = Orbits.ecef_orbitpos(orbit_pos, orbit_vel,dcm);


#get perpendicular baselines
bperp = Orbits.get_perp_baselines(orbit_pos, orbit_vel, look_angle);
bperp_ecef = Orbits.get_perp_baselines(pos_ecef, vel_ecef, look_angle);

#reshape data for plotting ease
bperp_all = reshape(bperp, size(bperp,1)*size(bperp,2), length(orbit_time))'/1e3;
bperp_ecef_all = reshape(bperp_ecef, size(bperp_ecef,1)*size(bperp_ecef,2), length(orbit_time))'/1e3;


#plot all perpendicular baselines and the max perpendicular baseline
p1 = plot(orbit_time, bperp_all, ylabel=("ECI (km)"), label ="", title="Perp Baseline")
p2 = plot(orbit_time, maximum(bperp_all, dims=2), label="", title=("Max Perp baseline"))

p3 = plot(orbit_time, bperp_ecef_all, xaxis=("time (sec)"), label ="", ylabel=("ECEF (km)"))
p4 = plot(orbit_time, maximum(bperp_ecef_all, dims=2),label="", xaxis=("time (sec)"))

plot(p1, p2, p3, p4, layout = (2,2))

# Compute variation in slant range over orbit for a certain look angle

In [ ]:
include("../modules/orbits.jl")
include("../modules/geometry.jl")
include("../modules/scene.jl")

using LinearAlgebra

look_angle = 35.0;

#compute look vectors for interpolated orbit
lookvec = zeros(size(pos_ecef))
slrng = zeros(size(pos_ecef,2), size(pos_ecef,3))


for ii=1:size(pos_ecef,2) # loop over platforms
    llh = Geometry.xyz_to_geo(pos_ecef[:,ii,:]);
    hdg = Geometry.compute_heading(llh[1,:]', llh[2,:]', vel_ecef[:,ii,:]);
    
    for jj=1:size(pos_ecef,3) # loop over timesteps
        
        #create peg point somewhere near the middle of the portion
        peg = Geometry.PegPoint(llh[1,jj], llh[2,jj], hdg[jj]);

        #compute cross-track range for the input look angle
        ρ,c = Scene.lookangle_to_range(look_angle,llh[3,jj],0, peg.Ra)

        #create point target
        pt = Geometry.sch_to_xyz([0,c,0], peg)
        
        #compute look vector and slant range
        lookvec[:,ii,jj] = pt - pos_ecef[:,ii,jj]; # look vector
        slrng[ii,jj] = norm(lookvec[:,ii,jj]); #slant range
    end
end

# plot look vector for platform 1
p1 = plot(orbit_time, lookvec[1,1,:]/1e3, ylabel=("X (km)"), label="", markersize=1)
p2 = plot(orbit_time, lookvec[2,1,:]/1e3, ylabel=("Y (km)"), label="", markersize=1)
p3 = plot(orbit_time, lookvec[3,1,:]/1e3, ylabel=("Z (km)"), xaxis=("time (sec)"),label="")
#plot slant ranges to all platforms
p4 = plot(orbit_time, slrng'/1e3, xaxis=("time (sec)"), 
    label="", title="Slant Ranges (km)")


plot(p1, p2,p3,p4, layout = @layout [[a{.9w};b{.9w};c{.9w}] d{.7w}])


# plot range variation within a SAR aperture for 1 point target

In [ ]:
include("../modules/orbits.jl")
include("../modules/geometry.jl")
include("../modules/scene.jl")

using LinearAlgebra

#chose portion of the orbit
fs = 5; #sampling rate.
duration = 5; #duration of orbit snippet
offset = 5.0; #offset to start of orbit snippet
tv = collect(0+offset:1/fs:duration+offset); #time vector
pegtvind = Int(round(length(tv)/2)); #where peg is going to be in the orbit
pegscind = 1; #which platform to use as peg reference

look_angle = 35.0;


#convert to ECEF
dcm = Orbits.eci_dcm(orbit_time, epoch);
pos, vel = Orbits.ecef_orbitpos(orbit_pos, orbit_vel,dcm);


#interpolate orbit
orbit_pos_interp = Orbits.interp_orbit(orbit_time, pos, tv);
orbit_vel_interp = Orbits.interp_orbit(orbit_time, vel, tv);


#create peg point somewhere near the middle of the portion
llh = Geometry.xyz_to_geo(pos[:,pegscind,pegtvind])
hdg = Geometry.compute_heading(llh[1], llh[2], vel[:,pegscind, pegtvind])
peg = Geometry.PegPoint(llh[1], llh[2], hdg[1]);

#compute cross-track range for the input look angle
ρ,c = Scene.lookangle_to_range(look_angle,llh[3],0, peg.Ra)

#create point target
pt = Geometry.sch_to_xyz([0,c,0], peg)

#compute look vectors for interpolated orbit
slrng = zeros(size(orbit_pos_interp,2), size(orbit_pos_interp,3))
lookvec = zeros(3,size(orbit_pos_interp,2), size(orbit_pos_interp,3))
for ii=1:size(orbit_pos_interp,2) # loop over platforms
    for jj=1:size(orbit_pos_interp,3) # loop over timesteps
        lookvec[:,ii,jj] = pt .-orbit_pos_interp[:,ii,jj]; # look vector
        slrng[ii,jj] = norm(lookvec[:,ii,jj]); #slant range
    end
end


#compute look vectors and slant range for original orbit without interpolation
ind = findall(x->x .>= tv[1] && x .<= tv[end], orbit_time);
lookvec_orig = pt .- pos #look vector
slrng_orig = mapslices(norm, reshape(lookvec_orig, 3, size(pos,2)*size(pos,3)), dims=1);
slrng_orig = reshape(slrng_orig, size(pos,2), size(pos,3));


# plot slant range XYZ
p1 = plot(tv, lookvec[1,1,:]/1e3, xaxis=("time (sec)"), label="",marker = (:dot), markersize=1)
p1 = plot!(orbit_time[ind], lookvec_orig[1,1,ind]/1e3, xaxis=("time (sec)"), 
    ylabel=("X (km)"), label="",marker = (:square), title="Look Vector, Sat 1, ECEF")
p2 = plot(tv, lookvec[2,1,:]/1e3, label="",marker = (:dot), markersize=1)
p2 = plot!(orbit_time[ind], lookvec_orig[2,1,ind]/1e3, xaxis=("time (sec)"), 
    ylabel=("Y (km)"), label="",marker = (:square))
p3 = plot(tv, lookvec[3,1,:]/1e3, label="",marker = (:dot), markersize=1)
p3 = plot!(orbit_time[ind], lookvec_orig[3,1,ind]/1e3, xaxis=("time (sec)"), 
    ylabel=("Z (km)"), label="",marker = (:square))
#plot slant range norm
p4 = plot(tv, slrng'/1e3, xaxis=("time (sec)"), 
    label="",marker = (:dot), markersize=1, title="Slant Ranges (km)")
p4 = plot!(orbit_time[ind], slrng_orig[:,ind]'/1e3, xaxis=("time (sec)"), 
    label="",marker = (:square))


plot(p1, p2,p3,p4, layout = @layout [[a{.9w};b{.9w};c{.9w}] d{.7w}])
